In [ ]:
import fitz  # PyMuPDF
import os

def extract_pdf_chunks(pdf_path, pages_per_chunk=5, overlap=1, output_dir="./chunks"):
    os.makedirs(output_dir, exist_ok=True)
    doc = fitz.open(pdf_path)
    total_pages = len(doc)

    start = 0
    chunk_id = 1
    while start < total_pages:
        end = min(start + pages_per_chunk, total_pages)
        text = ""
        for i in range(start, end):
            text += doc[i].get_text()
        with open(os.path.join(output_dir, f"chunk_{chunk_id:03}.txt"), "w", encoding="utf-8") as f:
            f.write(text)
        start = end - overlap
        chunk_id += 1
    doc.close()



    